# Utilisation du serveur de tracking MLflow du projet

Le serveur de tracking MLFlow est accessible à l'adresse suivante : https://parivision.heuzef.com:5000

Ce notebook explique comment utiliser ce dernier.

In [22]:
# Imports librairies et secrets
import mlflow
from mlflow.server import get_app_client
import requests
import setuptools
import os
from dotenv import load_dotenv
load_dotenv("../.env")
username = os.getenv('MLFLOW_TRACKING_USERNAME')
password = os.getenv('MLFLOW_TRACKING_PASSWORD')

In [23]:
# Connexion au serveur
mlflow_uri = "http://parivision.heuzef.com:5000" # L'adresse du serveur MLFlow'
client = get_app_client("basic-auth", tracking_uri=mlflow_uri)

print(f"Utilisateur : {client.get_user("parivision").username}")
print(f"Est admin : {client.get_user("parivision").is_admin}")

mlflow.set_tracking_uri(mlflow_uri)
mlflow.set_experiment("parivision") # Le nom du projet

Utilisateur : parivision
Est admin : True


<Experiment: artifact_location='mlflow-artifacts:/814265185531092965', creation_time=1741090978662, experiment_id='814265185531092965', last_update_time=1741090978662, lifecycle_stage='active', name='parivision', tags={}>

## Vérifier la disponibilité

Dans un premier temps, il faut s'assurer que le serveur est bien joignable.

In [24]:
# Effectuer une requête
response = requests.get(
    f"{mlflow_uri}/",
    auth=(username, password)
)

if response.status_code == 200:
    print("Le serveur de tracking MLflow est disponible ! Youpi ! ", mlflow_uri)
    print("Message :", response.text)
else:
    print(f"Échec de la connexion. Code d'erreur : {response.status_code}")
    print("Message :", response.text)


Le serveur de tracking MLflow est disponible ! Youpi !  http://parivision.heuzef.com:5000
Message : <!doctype html><html lang="en"><head><meta charset="utf-8"/><meta name="viewport" content="width=device-width,initial-scale=1,shrink-to-fit=no"/><link rel="shortcut icon" href="./static-files/favicon.ico"/><meta name="theme-color" content="#000000"/><link rel="manifest" href="./static-files/manifest.json" crossorigin="use-credentials"/><title>MLflow</title><script defer="defer" src="static-files/static/js/main.3e6a54ef.js"></script><link href="static-files/static/css/main.fd7f2361.css" rel="stylesheet"></head><body><noscript>You need to enable JavaScript to run this app.</noscript><div id="root"></div><div id="modal"></div></body></html>


# Entrainement d'un modèle pour l'exemple

Nous allons entrainer un petit modèle basique, avec Scikit-learn, pour obtenir quelques métriques qui seront enregistrés dans des variables.

In [25]:
# Imports librairies
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pandas as pd
import numpy as np

# Import d'une database (au pif juste pour ce test)
data = pd.read_csv("https://raw.githubusercontent.com/DataScientest/MLflow_Course/refs/heads/main/data/fake_data.csv")
X = data.drop(columns=["date", "demand"])
y = data["demand"]
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Train model
params = {
    "n_estimators": 10,
    "max_depth": 10,
    "random_state": 42,
}
rf = RandomForestRegressor(**params)
rf.fit(X_train, y_train)

# Evaluate model
y_pred = rf.predict(X_val)
mae = mean_absolute_error(y_val, y_pred)
mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_val, y_pred)
metrics = {"mae": mae, "mse": mse, "rmse": rmse, "r2": r2}

print("\nVoici des informations d'exemple que nous souhaiterions envoyer sur le serveur MLFlow :\n")

print("\nmetrics :")
print(metrics)

print("\nparams :")
print(params)


Voici des informations d'exemple que nous souhaiterions envoyer sur le serveur MLFlow :


metrics :
{'mae': 51.99824921798891, 'mse': 4387.742421525984, 'rmse': np.float64(66.24003639435884), 'r2': 0.8654539293255742}

params :
{'n_estimators': 10, 'max_depth': 10, 'random_state': 42}


## Envoi des informations au serveur MLflow

Maintenant que nous avons nos resultats, nous allons donc créer une "run" et la transférer sur le serveur. 

Pour cet exemple, c'est le module mlflow.sklearn qui est utilisé. Il vous faudra bien sur utiliser celui adapté à votre outil : https://mlflow.org/docs/latest/python_api/

In [ ]:
run_name = "test_002" # Le nom de la run, nous utiliserons notre propre nomenclature pour le projet

with mlflow.start_run(run_name=run_name) as run:
    mlflow.log_params(params)
    mlflow.log_metrics(metrics)
    mlflow.sklearn.log_model(sk_model=rf, input_example=X_val, artifact_path=run_name+"_artifacts")

/home/heuzef/GIT/DEC24_MLOPS_PARIS_SPORTIFS/.venv/lib/python3.12/site-packages/mlflow/types/utils.py:435: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


🏃 View run test_001 at: http://parivision.heuzef.com:5000/#/experiments/814265185531092965/runs/172c48b7a3eb455db61ba84b2011f70a
🧪 View experiment at: http://parivision.heuzef.com:5000/#/experiments/814265185531092965


Ce code executé en fin de script ou fin de notebook est finalement suffisant et assez flexible pour transférer les informations que nous souhaitons. Mais il est possible de faire encore plus simple en laissant MLflow se debrouiller avec `mlflow.autolog()`.

> https://mlflow.org/docs/latest/tracking/autolog.html

N'hésitez pas à tester des hyper-paramètres et envoyer quelques métriques à comparer sur l'interface.

# Charger le modèle le plus performant

> Ici, nous allons simplement charger la meilleur version du modèle, repéré grâce a son alias "Champion".
>
> http://parivision.heuzef.com:5000/#/models

In [28]:
champion = mlflow.pyfunc.load_model(f"models:/parivision@champion")

champion

mlflow.pyfunc.loaded_model:
  artifact_path: test_001_artifacts
  flavor: mlflow.sklearn
  run_id: 172c48b7a3eb455db61ba84b2011f70a

In [29]:
y_pred = champion.predict(X_val)
display(y_pred[:10])

array([1012.77094831, 1455.76527377,  931.17489626, 1327.67841297,
        951.93013   ,  896.20498834, 1186.90646238, 1251.68361962,
        933.77154864,  983.47213966])

In [31]:
from PIL import Image

artifact_uri="/mlartifacts"

with mlflow.start_run() as run:
    image = Image.new("RGB", (100, 100))
    artifact_uri = run.info.artifact_uri
    mlflow.log_image(image, "image.png")
    image = mlflow.artifacts.load_image(artifact_uri + "/image.png")
    print(image)


<PIL.PngImagePlugin.PngImageFile image mode=RGB size=100x100 at 0x7F75EF899BE0>
🏃 View run resilient-crab-523 at: http://parivision.heuzef.com:5000/#/experiments/814265185531092965/runs/5cd76ec0a0704577b8ce8e3637afebcd
🧪 View experiment at: http://parivision.heuzef.com:5000/#/experiments/814265185531092965
